In [1]:
import pandas as pd 
import numpy as np
import warnings

In [2]:
warnings.filterwarnings("ignore")

In [3]:
dataset = pd.read_csv("house_prices.csv")

In [4]:
dataset.shape

(187531, 15)

In [5]:
dataset.head(1)

,Title,Amount(in rupees),Price (in rupees),location,Carpet Area,Floor,Transaction,Furnishing,facing,overlooking,Bathroom,Balcony,Car Parking,Ownership,Super Area
0,1 BHK Ready to Occupy Flat for sale in Srushti...,42 Lac,6000.0,thane,500 sqft,10 out of 11,Resale,Unfurnished,NaN,NaN,1,2,NaN,NaN,NaN


In [6]:
dataset1 = dataset[dataset['Amount(in rupees)']=="Call for Price"]
dataset2 = dataset[dataset['Amount(in rupees)']!="Call for Price"]

In [7]:
def cr_to_lac(data):
    p = data.split(" ")
    if "Lac" in p[1]:
        return float(p[0])
    elif "Cr" in p[1]:
        return float(p[0])*100

In [8]:
dataset2["Amount(in rupees Lakhs )"] = dataset2['Amount(in rupees)'].apply(cr_to_lac)
dataset2.drop(columns=["Amount(in rupees)"],inplace=True)

In [9]:
for i in dataset1['location'].unique() :
    mean = dataset2[dataset2['location']==i]['Amount(in rupees Lakhs )'].mean()
    inx  = dataset1[dataset1['location']==i].index
    dataset1.loc[inx,["Amount(in rupees)"]] = mean

In [10]:
dataset1["Amount(in rupees Lakhs )"] = dataset1['Amount(in rupees)']
dataset1.drop(columns=['Amount(in rupees)'],inplace=True)

In [11]:
dataset = pd.concat((dataset1,dataset2),axis=0)

In [12]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Index: 187531 entries, 24 to 187530
Data columns (total 15 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   Title                     187531 non-null  object 
 1   Price (in rupees)         169866 non-null  float64
 2   location                  187531 non-null  object 
 3   Carpet Area               106858 non-null  object 
 4   Floor                     180454 non-null  object 
 5   Transaction               187448 non-null  object 
 6   Furnishing                184634 non-null  object 
 7   facing                    117298 non-null  object 
 8   overlooking               106095 non-null  object 
 9   Bathroom                  186703 non-null  object 
 10  Balcony                   138596 non-null  object 
 11  Car Parking               84174 non-null   object 
 12  Ownership                 122014 non-null  object 
 13  Super Area                79846 non-null   objec

In [13]:
dataset['Amount(in rupees Lakhs )'] = dataset['Amount(in rupees Lakhs )'].astype("float64")

In [14]:
def house_type(data):
    data = data.split(" ")
    return data[0]+" "+data[1]

In [15]:
dataset['House Type'] = dataset['Title'].apply(house_type)

In [16]:
dataset.head(2)

,Title,Price (in rupees),location,Carpet Area,Floor,Transaction,Furnishing,facing,overlooking,Bathroom,Balcony,Car Parking,Ownership,Super Area,Amount(in rupees Lakhs ),House Type
24,1 BHK Ready to Occupy Flat for sale Mumbra,NaN,thane,NaN,6 out of 7,Resale,Unfurnished,NaN,NaN,1,NaN,NaN,NaN,525 sqft,97.124933,1 BHK
37,1 BHK Ready to Occupy Flat for sale Kalyan East,NaN,thane,NaN,2 out of 4,Resale,Unfurnished,NaN,NaN,1,NaN,NaN,NaN,550 sqft,97.124933,1 BHK


In [17]:
dataset['Price (in rupees)'].unique()

array([   nan,  6000., 13799., ...,  2873.,  2663.,  2508.],
      shape=(10959,))

In [18]:
dataset['Price (in rupees)'].fillna(dataset['Price (in rupees)'].mean(),inplace=True)

In [19]:
dataset['Carpet Area'].unique()

array([nan, '550 sqft', '758 sqft', ..., '1634 sqft', '164 sqyrd',
       '136 sqft'], shape=(2759,), dtype=object)

In [20]:
# kanal , sqft , , marla , sqm , sqyrd  = sqft
# 1 kanal = 5445 sqft
# 1 marla  = 272.251 sqft
# 1 sqm = 10.7639 sqft
# 1 sqyrd = 9 sqft

In [21]:
def unit_change(data):
    t = type(data)
    if t == float :
        return data
    else :
        d = data.split(" ")
        if "kanal" in d[1]:
            return float(d[0])* 5445
        elif "marla" in d[1]:
            return float(d[0])*272.251
        elif "sqm" in d[1]:
            return float(d[0])*10.7639
        elif "sqyrd" in d[1]:
            return float(d[0])*9
        else :
            return float(d[0])

In [22]:
dataset['Carpet Area'] = dataset['Carpet Area'].apply(unit_change)

In [23]:
dataset['Super Area'] = dataset['Super Area'].str.replace(",","")
dataset['Super Area'] = dataset['Super Area'].apply(unit_change)

In [24]:
dataset = dataset.sort_index(axis=0)

In [25]:
dataset['Super Area'].fillna(0.0,inplace=True)

In [26]:
dataset["Area"] = dataset['Carpet Area'] + dataset['Super Area']

In [27]:
dataset['Area'].fillna(0.0,inplace=True)

In [28]:
for i in dataset['House Type'].unique():
    ma = dataset[(dataset['House Type']==i)&(dataset['Area']!=0.0)]['Area'].mean()
    inx = dataset[(dataset['House Type']==i)&(dataset['Area']==0.0)].index
    dataset.loc[inx,['Area']] = ma

In [29]:
dataset.isnull().sum()

Title                            0
Price (in rupees)                0
location                         0
Carpet Area                  80673
Floor                         7077
Transaction                     83
Furnishing                    2897
facing                       70233
overlooking                  81436
Bathroom                       828
Balcony                      48935
Car Parking                 103357
Ownership                    65517
Super Area                       0
Amount(in rupees Lakhs )         0
House Type                       0
Area                             7
dtype: int64

In [30]:
dataset.drop(columns=['Carpet Area',"Super Area"],inplace=True)

In [31]:
dataset['Ownership'].unique()

array([nan, 'Freehold', 'Co-operative Society', 'Power Of Attorney',
       'Leasehold'], dtype=object)

In [32]:
dataset['Furnishing'].fillna(dataset["Furnishing"].mode()[0],inplace=True)
dataset['facing'].fillna(dataset["facing"].mode()[0],inplace=True)
dataset['Bathroom'].fillna(dataset["Bathroom"].mode()[0],inplace=True)
dataset['Balcony'].fillna(0.0,inplace=True)
dataset['Ownership'].fillna(dataset["Ownership"].mode()[0],inplace=True)

In [33]:
dataset.isnull().sum()

Title                            0
Price (in rupees)                0
location                         0
Floor                         7077
Transaction                     83
Furnishing                       0
facing                           0
overlooking                  81436
Bathroom                         0
Balcony                          0
Car Parking                 103357
Ownership                        0
Amount(in rupees Lakhs )         0
House Type                       0
Area                             7
dtype: int64

In [34]:
dataset['Balcony'] = dataset['Balcony'].str.replace("> ","")
# dataset['Balcony'] = dataset['Balcony'].astype("float64")
# dataset['Balcony'] = dataset['Balcony'].astype("int64")

In [37]:
# dataset['Bathroom'] = dataset['Bathroom'].str.replace("> ","")
# dataset['Bathroom'] = dataset['Bathroom'].astype("float64")
dataset['Bathroom'] = dataset['Bathroom'].astype("int64")

In [38]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Index: 187531 entries, 0 to 187530
Data columns (total 15 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   Title                     187531 non-null  object 
 1   Price (in rupees)         187531 non-null  float64
 2   location                  187531 non-null  object 
 3   Floor                     180454 non-null  object 
 4   Transaction               187448 non-null  object 
 5   Furnishing                187531 non-null  object 
 6   facing                    187531 non-null  object 
 7   overlooking               106095 non-null  object 
 8   Bathroom                  187531 non-null  int64  
 9   Balcony                   138596 non-null  object 
 10  Car Parking               84174 non-null   object 
 11  Ownership                 187531 non-null  object 
 12  Amount(in rupees Lakhs )  187531 non-null  float64
 13  House Type                187531 non-null  object

In [39]:
dataset.isnull().sum()

Title                            0
Price (in rupees)                0
location                         0
Floor                         7077
Transaction                     83
Furnishing                       0
facing                           0
overlooking                  81436
Bathroom                         0
Balcony                      48935
Car Parking                 103357
Ownership                        0
Amount(in rupees Lakhs )         0
House Type                       0
Area                             7
dtype: int64

In [40]:
dataset.drop(columns=['Floor'],inplace=True)

In [41]:
dataset['Area'].fillna(dataset['Area'].mean(),inplace=True)

In [42]:
# Pool, Garden/Park, Main Road 
# Pool, Garden/Park
# Garden/Park, Main Road 
# Pool, Main Road 
# Not Available

In [43]:
l = ["Pool, Garden/Park, Main Road","Main Road, Garden/Park, Pool","Garden/Park, Pool, Main Road","Garden/Park, Main Road, Pool","Garden/Park, Pool, Main Road, Not Available","Main Road, Pool, Garden/Park", "Pool, Main Road, Garden/Park"]
dataset['overlooking'].replace(l,"Pool, Garden/Park, Main Road",inplace=True)
dataset['overlooking'] = dataset['overlooking'].str.replace(", Not Available","")
p = ["Main Road, Garden/Park","Garden/Park, Main Road"]
t = ["Garden/Park, Pool","Pool, Garden/Park"]
q = ["Pool, Main Road","Main Road, Pool"]
dataset['overlooking'].replace(p,"Main Road, Garden/Park",inplace=True)
dataset['overlooking'].replace(t,"Garden/Park, Pool",inplace=True)
dataset['overlooking'].replace(q,"Pool, Main Road",inplace=True)

In [44]:
for i in dataset['overlooking'].unique() :
    print(i)

nan
Garden/Park
Main Road, Garden/Park
Main Road
Pool, Garden/Park, Main Road
Garden/Park, Pool
Pool
Pool, Main Road


In [45]:
dataset['overlooking'].fillna(dataset['overlooking'].mode()[0],inplace=True)

In [46]:
dataset['Car Parking'] = dataset['Car Parking'].str.replace("Open","Covered")
dataset['Car Parking'] = dataset['Car Parking'].str.replace("Covered,","Covered")
dataset['Car Parking'].fillna("0",inplace= True)

In [47]:
f = lambda data : int(data.split(" ")[0])
dataset['Car Parking'] = dataset['Car Parking'].apply(f)

In [48]:
dataset['Transaction'].unique()

array(['Resale', 'New Property', nan, 'Other', 'Rent/Lease'], dtype=object)

In [49]:
dataset.dropna(inplace=True)

In [50]:
dataset.isnull().sum()

Title                       0
Price (in rupees)           0
location                    0
Transaction                 0
Furnishing                  0
facing                      0
overlooking                 0
Bathroom                    0
Balcony                     0
Car Parking                 0
Ownership                   0
Amount(in rupees Lakhs )    0
House Type                  0
Area                        0
dtype: int64

In [51]:
dataset.to_csv("house_prices_clean_part_1.csv",index=False)